# Deploying Iris-detection model using Vertex AI


## Overview

In this tutorial, you build a scikit-learn model and deploy it on Vertex AI using the custom container method. You use the FastAPI Python web server framework to create a prediction endpoint. You also incorporate a preprocessor from training pipeline into your online serving application.

Learn more about [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training) and [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions).

### Objective

In this notebook, you learn how to create, deploy and serve a custom classification model on Vertex AI. This notebook focuses more on deploying the model than on the design of the model itself.


This tutorial uses the following Vertex AI services and resources:

- Vertex AI models
- Vertex AI endpoints

The steps performed include:

- Train a model that uses flower's measurements as input to predict the class of iris.
- Save the model and its serialized pre-processor.
- Build a FastAPI server to handle predictions and health checks.
- Build a custom container with model artifacts.
- Upload and deploy custom container to Vertex AI Endpoints.

### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small and popular dataset for machine learning experiments. Each instance has four numerical features, which are different measurements of a flower, and a target label that
categorizes the flower into: **Iris setosa**, **Iris versicolour** and **Iris virginica**.

This tutorial uses [a version of the Iris dataset available in the
scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris).

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry
* Cloud Build

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing) and [Cloud Build pricing](https://cloud.google.com/build/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages



In [64]:

# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform

/opt/conda/envs/python_312_env/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=10880) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


### Set Google Cloud project information
Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [65]:
PROJECT_ID = "nice-carving-477506-q4"  # @param {type:"string"}
LOCATION = "asia-south2"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [66]:
BUCKET_URI = f"gs://mlops-wk8"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [67]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-wk8/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-wk8' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

In [68]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import the required libraries

In [69]:
import os
import sys
print(sys.version)

3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:25:55) [GCC 14.3.0]


### Configure resource names

Set a name for the following parameters:

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that is pushed to the repository.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI model resource.

In [70]:
MODEL_ARTIFACT_DIR = "artifacts"  # @param {type:"string"}
REPOSITORY = "iris-model-repo"  # @param {type:"string"}
IMAGE = "iris-model-img"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "iris-model"  # @param {type:"string"}

# Set the defaults if no names were specified
if MODEL_ARTIFACT_DIR == "[your-artifact-directory]":
    MODEL_ARTIFACT_DIR = "custom-artifacts"

if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-prediction"

if IMAGE == "[your-image-name]":
    IMAGE = "sklearn-fastapi-server"

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "sklearn-custom-container"

## Setup MLFlow

In [71]:
import mlflow
from mlflow import MlflowClient
from mlflow.models import infer_signature
from pprint import pprint

## Setup tracking

In [72]:
mlflow.set_tracking_uri("http://127.0.0.1:8100")
client = MlflowClient(mlflow.get_tracking_uri())
all_experiments = client.search_experiments()
print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/420563206152315427', creation_time=1762693701355, experiment_id='420563206152315427', last_update_time=1762693701355, lifecycle_stage='active', name='MLOps_Oppe_1: Mlflow Oppe experiment 1', tags={'mlflow.experimentKind': 'custom_model_development'}>, <Experiment: artifact_location='mlflow-artifacts:/540086182604046141', creation_time=1762610045881, experiment_id='540086182604046141', last_update_time=1762610045881, lifecycle_stage='active', name='MLOps_Oppe_Demo: Mlflow Oppe experiment Demo', tags={}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1762609294861, experiment_id='0', last_update_time=1762609294861, lifecycle_stage='active', name='Default', tags={}>]


In [73]:
mlflow.get_tracking_uri()

'http://127.0.0.1:8100'

In [74]:
mlflow.set_experiment("MLOps_Wk_8: Mlflow Data Poisoning")

2025/11/16 14:21:59 INFO mlflow.tracking.fluent: Experiment with name 'MLOps_Wk_8: Mlflow Data Poisoning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/680739964798949896', creation_time=1763302919780, experiment_id='680739964798949896', last_update_time=1763302919780, lifecycle_stage='active', name='MLOps_Wk_8: Mlflow Data Poisoning', tags={}>

## Simple Decision Tree model
Build a Decision Tree model on iris data

In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

data = pd.read_csv(BUCKET_URI+'/data/iris.csv')
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.3,3.4,1.9,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,4.9,3.7,1.0,0.4,setosa


In [99]:
train, test = train_test_split(data, test_size = 0.4, stratify = data['species'], random_state = 42)
X_train = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train = train.species
X_test = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test = test.species

In [100]:
params= { 
    "max_depth": 5,
    "random_state":10
}
model_dt = DecisionTreeClassifier(**params)
model_dt.fit(X_train,y_train)
prediction=model_dt.predict(X_test)
accuracy_score = metrics.accuracy_score(prediction,y_test)
print('The accuracy of the Decision Tree is',"{:.3f}".format(accuracy_score))

The accuracy of the Decision Tree is 0.917


In [101]:
X_train.head()

,sepal_length,sepal_width,petal_length,petal_width
130,7.4,2.8,6.1,1.9
6,4.6,3.4,1.4,0.3
94,5.5,2.8,4.0,1.4
96,5.6,2.9,4.4,1.2
26,5.1,3.3,1.7,0.5


In [102]:
import pickle
import joblib
import gcsfs

fs = gcsfs.GCSFileSystem()
with fs.open( BUCKET_URI+"/artifacts/iris_model.joblib", 'wb') as f:
    joblib.dump(model_dt, f)



## Write data to MLFlow

In [103]:
with mlflow.start_run():
    mlflow.log_params(params)
    
    mlflow.log_metric("accuracy", accuracy_score)
    
    mlflow.set_tag("Training Info","Iris Data-MLOps")
    
    signature=infer_signature(X_train, model_dt.predict(X_train))

    model_info = mlflow.sklearn.log_model(
        sk_model = model_dt,
        signature = signature,
        input_example = X_train,
        artifact_path = "iris_data",
        registered_model_name= "iris_model_dt",
    )

2025/11/16 14:45:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'iris_model_dt' already exists. Creating a new version of this model...
2025/11/16 14:45:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_model_dt, version 4
Created version '4' of model 'iris_model_dt'.


🏃 View run crawling-tern-258 at: http://127.0.0.1:8100/#/experiments/680739964798949896/runs/d334cb697e39436e800f5b5c3f687671
🧪 View experiment at: http://127.0.0.1:8100/#/experiments/680739964798949896


### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

* `model.joblib` (model artifact)
* `preprocessor.pkl` (model artifact)

Run the following commands to upload your files:

In [ ]:
!gsutil cp artifacts/model.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://artifacts/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][  2.6 KiB/  2.6 KiB]                                                
Operation completed over 1 objects/2.6 KiB.                                      
